In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

df = pd.read_csv('/Users/julialau/Downloads/ads_dataset.csv', index_col=False)


### Clean Data

In [2]:
df.head()

,wp_identifier,archive_id,embed_url,sponsors,sponsor_types,subjects,candidates,type,race,cycle,message,air_count,reference_count,market_count,transcript,date_ingested
0,8403,PolAd_CatherineCortezMasto_jbah8,https://archive.org/embed/PolAd_CatherineCorte...,Catherine Cortez Masto for Senate,Candidate Committee,"Candidate Biography, Bipartisanship, Criminal ...",Catherine Cortez Masto,campaign,NVS2,"2,016.00",pro,185,0,1,NaN,2016/11/17 5:03:54 UTC
1,8404,PolAd_DonaldTrump_kc0en,https://archive.org/embed/PolAd_DonaldTrump_kc0en,Hillary for America,Multiple,"Nuclear, Candidate Biography, Military, Foreig...",Donald Trump,campaign,PRES,"2,016.00",con,139,0,1,NaN,2016/11/17 5:02:52 UTC
2,8358,PolAd_DonaldTrump_ncck5,https://archive.org/embed/PolAd_DonaldTrump_ncck5,Hillary for America,Multiple,"Bankruptcy, Candidate Biography, Workers",Donald Trump,campaign,PRES,"2,016.00",con,195,0,2,NaN,2016/11/14 10:44:42 UTC
3,8359,PolAd_Guns_gv0de,https://archive.org/embed/PolAd_Guns_gv0de,National Rifle Assn,PAC,Guns,NaN,campaign,NaN,NaN,unknown,87,0,3,NaN,2016/11/14 10:44:15 UTC
4,8361,PolAd_Guns_Veterans_Military_z2bvd,https://archive.org/embed/PolAd_Guns_Veterans_...,National Rifle Assn,PAC,"Guns, Veterans, Military",NaN,campaign,NaN,NaN,unknown,54,0,3,NaN,2016/11/14 10:43:51 UTC


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2842 entries, 0 to 2841
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   wp_identifier    2842 non-null   int64  
 1   archive_id       2842 non-null   object 
 2   embed_url        2842 non-null   object 
 3   sponsors         2838 non-null   object 
 4   sponsor_types    2838 non-null   object 
 5   subjects         2678 non-null   object 
 6   candidates       2730 non-null   object 
 7   type             2842 non-null   object 
 8   race             2696 non-null   object 
 9   cycle            2701 non-null   float64
 10  message          2842 non-null   object 
 11  air_count        2842 non-null   int64  
 12  reference_count  2842 non-null   int64  
 13  market_count     2842 non-null   int64  
 14  transcript       1365 non-null   object 
 15  date_ingested    2842 non-null   object 
dtypes: float64(1), int64(4), object(11)
memory usage: 355.4+ KB


In [4]:
df.shape

(2842, 16)

In [5]:
#check for missing values
df.isnull().sum()

wp_identifier         0
archive_id            0
embed_url             0
sponsors              4
sponsor_types         4
subjects            164
candidates          112
type                  0
race                146
cycle               141
message               0
air_count             0
reference_count       0
market_count          0
transcript         1477
date_ingested         0
dtype: int64

In [6]:
df.describe()

,wp_identifier,cycle,air_count,reference_count,market_count
count,"2,842.00","2,701.00","2,842.00","2,842.00","2,842.00"
mean,"3,896.28","2,016.00",132.07,0.05,1.45
std,"2,417.30",0.00,344.19,0.24,2.95
min,232.00,"2,016.00",0.00,0.00,0.00
25%,"1,630.00","2,016.00",0.00,0.00,0.00
50%,"3,768.50","2,016.00",0.00,0.00,1.00
75%,"5,912.50","2,016.00",131.00,0.00,1.00
max,"8,404.00","2,016.00","5,084.00",4.00,23.00


In [7]:
# check for duplicate rows
df.duplicated().sum()

0

In [8]:
columns = ['transcript'] # more than half of the rows have null for this
df = df.drop(columns=columns)

In [9]:
# columns = ['candidates', 'subjects', 'race', 'cycle']
# df[columns] = df[columns].fillna('Unknown') #fill nulls with 'Unknown'

In [15]:
columns = ['candidates', 'subjects', 'race', 'cycle', 'sponsor_types', 'sponsors']
df = df.dropna(subset=columns)

In [16]:
df['date_ingested'] = pd.to_datetime(df['date_ingested'], errors='coerce') #cover to datetime

In [17]:
df.shape

(2530, 15)

In [18]:
print(df['sponsor_types'].unique())
print(df['type'].unique())
print(df['race'].unique())

['Candidate Committee' 'Multiple' 'Hybrid Super PAC' 'PAC' 'Super PAC'
 'Non Profit' 'Trade Association' 'Unknown' 'Joint Fundraising Committee'
 'Corporation' '527' 'c4']
['campaign']
['NVS2' 'PRES' 'WIS2' 'AZS1' 'NHS1' 'PAS1' 'FLS2' 'IAS1' 'NCS2' 'PA08'
 'NJ03' 'COS1' 'OHS2' 'INS2' 'ILS2' 'CAS1' 'UT04' 'FL26' 'WI01' 'MSS1'
 'IAS2' 'SCS1' 'AZ05' 'ME02' 'KS01' 'CO06' 'AKS2' 'NV03' 'NV04' 'FL02'
 'MOS1' 'KYS2' 'NJ05' 'NY01' 'LA02' 'NC13' 'NC02' 'NC03' 'FL13' 'CA24'
 'MD08' 'VA02' 'IA01' 'GAS2' 'IN03' 'MDS2' 'MD04' 'NY19' 'PA02' 'OH08'
 'FL06' 'SCS2' 'OH14' 'IA03' 'UTS2' 'NC11' 'VA07' 'MD06' 'KY04' 'DE01']


In [19]:
df.head()

,wp_identifier,archive_id,embed_url,sponsors,sponsor_types,subjects,candidates,type,race,cycle,message,air_count,reference_count,market_count,date_ingested
0,8403,PolAd_CatherineCortezMasto_jbah8,https://archive.org/embed/PolAd_CatherineCorte...,Catherine Cortez Masto for Senate,Candidate Committee,"Candidate Biography, Bipartisanship, Criminal ...",Catherine Cortez Masto,campaign,NVS2,"2,016.00",pro,185,0,1,2016-11-17 05:03:54+00:00
1,8404,PolAd_DonaldTrump_kc0en,https://archive.org/embed/PolAd_DonaldTrump_kc0en,Hillary for America,Multiple,"Nuclear, Candidate Biography, Military, Foreig...",Donald Trump,campaign,PRES,"2,016.00",con,139,0,1,2016-11-17 05:02:52+00:00
2,8358,PolAd_DonaldTrump_ncck5,https://archive.org/embed/PolAd_DonaldTrump_ncck5,Hillary for America,Multiple,"Bankruptcy, Candidate Biography, Workers",Donald Trump,campaign,PRES,"2,016.00",con,195,0,2,2016-11-14 10:44:42+00:00
5,8364,PolAd_RussFeingold_ey28u,https://archive.org/embed/PolAd_RussFeingold_e...,Russ For Wisconsin,Candidate Committee,"Medicare, Jobs, Trade",Russ Feingold,campaign,WIS2,"2,016.00",unknown,276,0,1,2016-11-14 10:43:27+00:00
7,8303,PolAd_JohnMccain_005nx,https://archive.org/embed/PolAd_JohnMccain_005nx,Friends of John McCain,Candidate Committee,"Campaign Finance, Military, Ethics, Federal Bu...",John McCain,campaign,AZS1,"2,016.00",pro,14,0,1,2016-11-11 01:53:06+00:00


### Start by establishing top n (3) candidates with highest air time
For this analysis, we will focus primarily on the top 3 candidates: 
1. Donald Trump
2. Hilary Clinton
3. Bernie Sanders

In [82]:
import plotly.express as px

top_candidates = df.groupby('candidates')['air_count'].sum().sort_values(ascending=False).head(3).reset_index()

color_map = {
    'Donald Trump': '#F6A5A1', 
    'Hillary Clinton': '#A6C9E2',
    'Bernie Sanders': '#BFDAA4',
    'Unknown': '#808080'
}

fig = px.bar(top_candidates, 
             x='air_count', 
             y='candidates', 
             color='candidates', 
             color_discrete_map=color_map,
             labels={'air_count': 'Total Air Count', 'candidates': 'Candidates'},
             title='Top 3 Candidates with Highest Airtime')

fig.update_layout(showlegend=False)
fig.show()


### Content: what are the top subjects in their ads? 

For Trump, he mainly focused on: 
1. Candidate Biography
2. Immigration (he was the only one who had ads in this area)
3. Women


For Hilary, she mainly focused on: 
1. Candidate Biography
2. Women 
3. Foreign Policy (she was the only one who had ads in this area)

For Bernie, he mainly focused on: 
1. Economy 
2. Campaign Finance (idk what this means)
3. Education 
--> his subjects overlapped the least with the other two candidates

In [89]:
import plotly.express as px

top_candidates = ['Donald Trump', 'Hillary Clinton', 'Bernie Sanders']
filtered_df = df[df['candidates'].isin(top_candidates)]

# split the subjects column into individual rows
filtered_exploded = filtered_df.assign(subjects=filtered_df['subjects'].str.split(', ')).explode('subjects')

# group by candidates and subjects, then get the top 5 subjects by count
top_subjects = filtered_exploded.groupby('candidates')['subjects'].value_counts().groupby(level=0).head(5).reset_index(name='count')

fig = px.bar(top_subjects,
             x='count',
             y='subjects',
             color='candidates',
             title='Top Subjects in Ads for Top 3 Candidates',
             labels={'count': 'Count', 'subjects': 'Subjects', 'candidates': 'Candidates'},
             color_discrete_map=color_map)

fig.update_layout(barmode='stack', xaxis_title='Count', yaxis_title='Subjects', legend_title='Candidates')
fig.show()


In [90]:
fig = px.bar(top_subjects,
             x='count',
             y='subjects',
             color='candidates',
             facet_row='candidates',  #facet by candidates (separate rows for each)
             title='Top Subjects in Ads for Top 3 Candidates',
             labels={'count': 'Count', 'subjects': 'Subjects', 'can': 'Candidates'},
             color_discrete_map=color_map)

fig.update_layout(
    xaxis_title='Count', 
    yaxis_title='Subjects', 
    legend_title='Candidates',
    showlegend=False
)

fig.show()


### Tone: supportive/pro vs attack/negative ads

Negative ads made up 75% of Trump's ads

Positive ads made up 62% of Hillary's ads 

Unknown ads made up 60% of Bernie's ads

In [86]:
tone_analysis = filtered_df.groupby(['candidates', 'message'])['air_count'].sum().unstack(fill_value=0)

# normalize the data to get percentages
tone_percentage = tone_analysis.div(tone_analysis.sum(axis=1), axis=0) * 100

tone_colors = {
    'con': '#F6A5A1', 
    'mixed': '#A6C9E2', 
    'pro': '#BFDAA4', 
    'unknown': '#808080' 
}

fig = px.bar(tone_percentage,
             x=tone_percentage.index,
             y=tone_percentage.columns,
             title='Tone Analysis of Ads for Top 3 Candidates',
             labels={'value': 'Percentage', 'candidates': 'Candidates', 'message': 'Tone'},
             color_discrete_map=tone_colors,
             category_orders={'message': ['con', 'mixed', 'pro', 'unknown']})

fig.update_layout(barmode='stack', xaxis_title='Candidates', yaxis_title='Percentage', legend_title='Message')
fig.show()


### Geographic Reach: which candidates target the most markets, how tone differs across these markets

Donald trump had the largest reach

In [91]:
import plotly.express as px

geo_analysis = filtered_df.groupby(['candidates', 'message'])['market_count'].sum().unstack(fill_value=0)

geo_analysis = geo_analysis.reset_index()

fig = px.bar(geo_analysis,
             x='candidates',
             y=geo_analysis.columns[1:],  # all columns except 'candidates'
             title='Geographic Reach & Tone of Ads for Top 3 Candidates',
             labels={'value': 'Market Count', 'candidates': 'Candidates', 'message': 'Message'},
             color_discrete_map=tone_colors,
             barmode='stack')

fig.update_layout(xaxis_title='Candidates', yaxis_title='Market Count', legend_title='Message')
fig.show()


In [34]:
geo_analysis.head()

message,con,mixed,pro,unknown
candidates,,,,
Bernie Sanders,4,0,109,114
Donald Trump,581,3,184,21
Hillary Clinton,320,0,355,63


### Sponsor Types: compare how PACs, candidate committees, or other sponsors types use air time


In [88]:
sponsor_colors = {
    'Candidate Committee': '#F9D6B0',
    'PAC': '#D8E7F5',
    'Multiple': '#D3E9D3',
    'Super PAC': '#F4A1C7',
    'Other': '#E2D0F7',
    'Corporation': '#C0D9E9',
    'Non Profit': '#D9E8D8',
    'Hybrid Super PAC': '#F1D5D8',
    'Unknown': '#B0B0B0'
}

sponsor_analysis = filtered_df.groupby(['candidates', 'sponsor_types'])['air_count'].sum().unstack(fill_value=0)

fig = px.bar(sponsor_analysis,
             x=sponsor_analysis.index,
             y=sponsor_analysis.columns,
             title='Sponsor Types for Top 3 Candidates',
             labels={'value': 'Air Count', 'candidates': 'Candidates', 'sponsor_types': 'Sponsor Type'},
             color_discrete_map=sponsor_colors)

fig.update_layout(barmode='stack', xaxis_title='Candidates', yaxis_title='Air Count', legend_title='Sponsor Type')
fig.show()
